## Definicion de herramientas:

 - fetch_flight_info: get flight information for certain dates.
 - pick_flight: pick the best flight based on some criteria.
 - book_flight: book a flight on behalf of the user.
 - fetch_itinerary: get the information of a booked itinerary.
 - cancel_itinerary: cancel a booked itinerary.
 - get_user_info: get users' information.
 - file_ticket: file a backlog ticket to have human assist.

## Data Structure

In [ ]:
from pydantic import BaseModel

class Date(BaseModel):
    year: int
    month: int
    day: int
    hour: int

class UserProfile(BaseModel):
    user_id: str
    name: str
    email: str

class Flight(BaseModel):
    flight_id: str
    date_time: Date
    origin: str
    destination: str
    duration: float
    price: float

class Itinerary(BaseModel):
    confirmation_number: str
    user_profile: UserProfile
    flight: Flight

class Ticket(BaseModel):
    user_request: str
    user_profile: UserProfile

## Datos de prueba

In [ ]:
user_database = {
    "Adam": UserProfile(user_id="1", name="Adam", email="adam@gmail.com"),
    "Bob": UserProfile(user_id="2", name="Bob", email="bob@gmail.com"),
    "Chelsie": UserProfile(user_id="3", name="Chelsie", email="chelsie@gmail.com"),
    "David": UserProfile(user_id="4", name="David", email="david@gmail.com"),
}

flight_database = {
    "DA123": Flight(
        flight_id="DA123",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=1),
        duration=3,
        price=200,
    ),
    "DA125": Flight(
        flight_id="DA125",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=7),
        duration=9,
        price=500,
    ),
    "DA456": Flight(
        flight_id="DA456",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=1),
        duration=2,
        price=100,
    ),
    "DA460": Flight(
        flight_id="DA460",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=9),
        duration=2,
        price=120,
    ),
    "BO470": Flight(
        flight_id="BO470",
        origin="LPB",
        destination="CBB",
        date_time=Date(year=2025, month=10, day=31, hour=14),
        duration=1,
        price=20,
    ),
}

itinery_database = {}
ticket_database = {}

## Definir las herramientas

Es importante definir cada herramienta (funcion) con:

 - Un docstring que defina la tarea de la herramienta, tambien agregar un nombre de función descriptivo.
 - Agregar type hints para los argumentos.

In [ ]:
import random
import string


def fetch_flight_info(date: Date, origin: str, destination: str):
    """Fetch flight information from origin to destination on the given date"""
    flights = []

    for flight_id, flight in flight_database.items():
        if (
            flight.date_time.year == date.year
            and flight.date_time.month == date.month
            and flight.date_time.day == date.day
            and flight.origin == origin
            and flight.destination == destination
        ):
            flights.append(flight)
    if len(flights) == 0:
        raise ValueError("No matching flight found!")
    return flights


def fetch_itinerary(confirmation_number: str):
    """Fetch a booked itinerary information from database"""
    return itinery_database.get(confirmation_number)


def pick_flight(flights: list[Flight]):
    """Pick up the best flight that matches users' request. we pick the shortest, and cheaper one on ties."""
    sorted_flights = sorted(
        flights,
        key=lambda x: (
            x.get("duration") if isinstance(x, dict) else x.duration,
            x.get("price") if isinstance(x, dict) else x.price,
        ),
    )
    return sorted_flights[0]


def _generate_id(length=8):
    chars = string.ascii_lowercase + string.digits
    return "".join(random.choices(chars, k=length))


def book_flight(flight: Flight, user_profile: UserProfile):
    """Book a flight on behalf of the user."""
    confirmation_number = _generate_id()
    while confirmation_number in itinery_database:
        confirmation_number = _generate_id()
    itinery_database[confirmation_number] = Itinerary(
        confirmation_number=confirmation_number,
        user_profile=user_profile,
        flight=flight,
    )
    return confirmation_number, itinery_database[confirmation_number]


def cancel_itinerary(confirmation_number: str, user_profile: UserProfile):
    """Cancel an itinerary on behalf of the user."""
    if confirmation_number in itinery_database:
        del itinery_database[confirmation_number]
        return
    raise ValueError("Cannot find the itinerary, please check your confirmation number.")


def get_user_info(name: str):
    """Fetch the user profile from database with given name."""
    return user_database.get(name)


def file_ticket(user_request: str, user_profile: UserProfile):
    """File a customer support ticket if this is something the agent cannot handle."""
    ticket_id = _generate_id(length=6)
    ticket_database[ticket_id] = Ticket(
        user_request=user_request,
        user_profile=user_profile,
    )
    return ticket_id

## Crear un agente ReAct

Un agente ReAct es un patron que combina razonamiento y acción. Se usa el patrón de chain of thought (CoT) para razonamiento en conjunto con el patrón de uso de herramientas. Este enfoque mejora las capacidades de un LLM para gestionar y resolver problemas complejos y toma de decisiones

In [19]:
import dspy

class DSPyAirlineCustomerService(dspy.Signature):
    """You are an airline customer service agent that helps user book and manage flights.

    You are given a list of tools to handle user request, and you should decide the right tool to use in order to
    fulfill users' request."""

    chat_history: str = dspy.InputField(desc="The precious user messages and agent responses")
    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(
        desc=(
                "Message that summarizes the process result, and the information users need, e.g., the "
                "confirmation_number if a new flight is booked."
            )
        )

In [20]:
agent = dspy.ReAct(
    DSPyAirlineCustomerService,
    tools = [
        fetch_flight_info,
        fetch_itinerary,
        pick_flight,
        book_flight,
        cancel_itinerary,
        get_user_info,
        file_ticket,
    ]
)

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

In [24]:
chat_history = ""

In [25]:
# turno 1
user_request = "please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam"
result = agent(user_request=user_request, chat_history=chat_history)
print(result)

Prediction(
    trajectory={'thought_0': 'I need to fetch flight information for Adam from SFO to JFK on 09/01/2025 to find available flights for booking.', 'tool_name_0': 'fetch_flight_info', 'tool_args_0': {'date': {'year': 2025, 'month': 9, 'day': 1, 'hour': 0}, 'origin': 'SFO', 'destination': 'JFK'}, 'observation_0': [Flight(flight_id='DA123', date_time=Date(year=2025, month=9, day=1, hour=1), origin='SFO', destination='JFK', duration=3.0, price=200.0), Flight(flight_id='DA125', date_time=Date(year=2025, month=9, day=1, hour=7), origin='SFO', destination='JFK', duration=9.0, price=500.0)], 'thought_1': 'I have retrieved the available flights from SFO to JFK on 09/01/2025. The options are flight DA123, which departs at 1 AM and costs $200, and flight DA125, which departs at 7 AM and costs $500. I will now pick the best flight for Adam based on the shortest duration and lowest price.', 'tool_name_1': 'pick_flight', 'tool_args_1': {'flights': [{'flight_id': 'DA123', 'date_time': {'yea

In [26]:
print(itinery_database)

{'h9iheltf': Itinerary(confirmation_number='h9iheltf', user_profile=UserProfile(user_id='1', name='Adam', email='adam@gmail.com'), flight=Flight(flight_id='DA123', date_time=Date(year=2025, month=9, day=1, hour=1), origin='SFO', destination='JFK', duration=3.0, price=200.0)), 'hn0abley': Itinerary(confirmation_number='hn0abley', user_profile=UserProfile(user_id='1', name='Adam', email='adam@gmail.com'), flight=Flight(flight_id='DA125', date_time=Date(year=2025, month=9, day=1, hour=7), origin='SFO', destination='JFK', duration=9.0, price=500.0)), 'fz2l7nau': Itinerary(confirmation_number='fz2l7nau', user_profile=UserProfile(user_id='1', name='Adam', email='adam@gmail.com'), flight=Flight(flight_id='DA123', date_time=Date(year=2025, month=9, day=1, hour=1), origin='SFO', destination='JFK', duration=3.0, price=200.0))}


In [ ]:
dspy.inspect_history(n=10)

In [30]:
chat_history += f"User: {user_request}\nAgent: {result.process_result}\n"

confirmation_number = "h9iheltf"

user_request = f"i  want to talk with a human please"

result = agent(user_request=user_request, chat_history=chat_history)
print(result.process_result)

A customer support ticket has been successfully filed for your request to talk with a human. You will be contacted shortly by a representative.


In [31]:
ticket_database

{'telfzn': Ticket(user_request='i want to talk with a human please', user_profile=UserProfile(user_id='Adam', name='Adam', email=''))}

In [ ]:
result = agent(user_request=f"yes, i confirm the operation")
print(result.process_result)